In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import yaml
import warnings
import datetime
import sklearn
warnings.filterwarnings("ignore")
plt.style.use('ggplot')

### collect data from CSV

https://www.kaggle.com/manishkc06/startup-success-prediction

In [2]:
startup = pd.read_csv('data//in//startup_data.csv')

In [3]:
startup.head(5)

,unamed:0,state_code,latitude,longitude,zip_code,id,city,Unnamed: 6,name,labels,...,object_id,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,is_top500,status
0,1005,CA,42.358880,-71.056820,92101,c:6669,San Diego,NaN,Bandsintown,1.0,...,c:6669,0.0,1.0,0.0,0.0,0.0,0.0,1.0000,0.0,acquired
1,204,CA,37.238916,-121.973718,95032,c:16283,Los Gatos,NaN,TriCipher,1.0,...,c:16283,1.0,0.0,0.0,1.0,1.0,1.0,4.7500,1.0,acquired
2,1001,CA,32.901049,-117.192656,92121,c:65620,San Diego,San Diego CA 92121,Plixi,1.0,...,c:65620,0.0,0.0,1.0,0.0,0.0,0.0,4.0000,1.0,acquired
3,738,CA,37.320309,-122.050040,95014,c:42668,Cupertino,Cupertino CA 95014,Solidcore Systems,1.0,...,c:42668,0.0,0.0,0.0,1.0,1.0,1.0,3.3333,1.0,acquired
4,1002,CA,37.779281,-122.419236,94105,c:65806,San Francisco,San Francisco CA 94105,Inhale Digital,0.0,...,c:65806,1.0,1.0,0.0,0.0,0.0,0.0,1.0000,1.0,closed


In [4]:
startup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923 entries, 0 to 922
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   unamed:0                  923 non-null    object 
 1   state_code                915 non-null    object 
 2   latitude                  915 non-null    float64
 3   longitude                 915 non-null    float64
 4   zip_code                  915 non-null    object 
 5   id                        915 non-null    object 
 6   city                      915 non-null    object 
 7   Unnamed: 6                428 non-null    object 
 8   name                      915 non-null    object 
 9   labels                    915 non-null    float64
 10  founded_at                915 non-null    object 
 11  closed_at                 333 non-null    object 
 12  first_funding_at          915 non-null    object 
 13  last_funding_at           915 non-null    object 
 14  age_first_

# Data cleaning

We take just the columns that could be used in a ML algorithm

In [5]:
startup_clean = startup[['name','age_first_funding_year','age_last_funding_year','age_first_milestone_year','age_last_milestone_year', 'relationships','funding_rounds', 'funding_total_usd','is_CA','is_TX','is_NY'
         ,'is_MA', 'is_otherstate', 'is_software', 'is_web', 'is_mobile', 'is_enterprise','is_advertising','is_gamesvideo','is_ecommerce','is_biotech','is_consulting','is_othercategory','has_angel',
        'has_VC','has_roundA','has_roundB','has_roundC','has_roundD','avg_participants', 'status']]

We convert the 'status', that will be our Y to predict, in a boolean variable

In [6]:
startup_clean['status'] = startup_clean['status'].replace('acquired', 1)
startup_clean['status'] = startup_clean['status'].replace('closed', 0)

startup_clean = startup_clean.rename(columns={'status': 'has_succeded'})

In [7]:
startup_clean = startup_clean[startup_clean['has_succeded'].notna()]

In [58]:
startup_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 915 entries, 0 to 922
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      915 non-null    object 
 1   age_first_funding_year    915 non-null    float64
 2   age_last_funding_year     915 non-null    float64
 3   age_first_milestone_year  915 non-null    float64
 4   age_last_milestone_year   915 non-null    float64
 5   relationships             915 non-null    float64
 6   funding_rounds            915 non-null    float64
 7   funding_total_usd         915 non-null    float64
 8   is_CA                     915 non-null    float64
 9   is_TX                     915 non-null    float64
 10  is_NY                     915 non-null    float64
 11  is_MA                     915 non-null    float64
 12  is_otherstate             915 non-null    float64
 13  is_software               915 non-null    float64
 14  is_web    

In [20]:
startup_clean#[startup['age_first_milestone_year'].isna()][['milestones','age_first_milestone_year','age_last_milestone_year']]

,name,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,funding_rounds,funding_total_usd,is_CA,is_TX,...,is_consulting,is_othercategory,has_angel,has_VC,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,has_succeded
0,Bandsintown,2.2493,3.0027,4.6685,6.7041,3.0,3.0,375000.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0000,1.0
1,TriCipher,5.1260,9.9973,7.0055,7.0055,9.0,4.0,40100000.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,4.7500,1.0
2,Plixi,1.0329,1.0329,1.4575,2.2055,5.0,1.0,2600000.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0000,1.0
3,Solidcore Systems,3.1315,5.3151,6.0027,6.0027,5.0,3.0,40000000.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.3333,1.0
4,Inhale Digital,0.0000,1.6685,0.0384,0.0384,2.0,2.0,1300000.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,CoTweet,0.5178,0.5178,0.5808,4.5260,9.0,1.0,1100000.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0000,1.0
919,Reef Point Systems,7.2521,9.2274,6.0027,6.0027,1.0,3.0,52000000.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.6667,0.0
920,Paracor Medical,8.4959,8.4959,9.0055,9.0055,5.0,1.0,44000000.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0000,0.0
921,Causata,0.7589,2.8329,0.7589,3.8356,12.0,2.0,15500000.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0000,1.0


in order to avoid the null value on the fields 'age_first_milestone_year' and 'age_last_milestone_year' 
we procedeed in this way: <br>
    - calculate the mean of these value <br>
    - replace the nan with the mean

In [10]:
age_fmy_mean = startup_clean['age_first_milestone_year'].mean()
age_lmy_mean = startup_clean['age_last_milestone_year'].mean()

startup_clean['age_first_milestone_year']= startup_clean['age_first_milestone_year'].fillna(age_fmy_mean)
startup_clean['age_last_milestone_year']= startup_clean['age_last_milestone_year'].fillna(age_fmy_mean)

check on null values

In [11]:
startup_clean.isnull().sum()

name                        0
age_first_funding_year      0
age_last_funding_year       0
age_first_milestone_year    0
age_last_milestone_year     0
relationships               0
funding_rounds              0
funding_total_usd           0
is_CA                       0
is_TX                       0
is_NY                       0
is_MA                       0
is_otherstate               0
is_software                 0
is_web                      0
is_mobile                   0
is_enterprise               0
is_advertising              0
is_gamesvideo               0
is_ecommerce                0
is_biotech                  0
is_consulting               0
is_othercategory            0
has_angel                   0
has_VC                      0
has_roundA                  0
has_roundB                  0
has_roundC                  0
has_roundD                  0
avg_participants            0
has_succeded                0
dtype: int64

# Kfold validation

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

function to get score of a model in a simpler way

In [13]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return fit.score(X_test, y_test)

In [14]:
X = startup_clean[['age_first_funding_year','age_last_funding_year','age_first_milestone_year','age_last_milestone_year', 'relationships','funding_rounds', 'funding_total_usd','is_CA','is_TX','is_NY'
        ,'is_MA', 'is_otherstate', 'is_software', 'is_web', 'is_mobile', 'is_enterprise','is_advertising','is_gamesvideo','is_ecommerce','is_biotech','is_consulting','is_othercategory','has_angel',
       'has_VC','has_roundA','has_roundB','has_roundC','has_roundD','avg_participants']].to_numpy()

#X1 =startup_clean[['age_first_funding_year', 'age_last_funding_year']]

y = startup_clean['has_succeded'].to_numpy()

In [15]:
print(X.shape)
print(y.shape)

(915, 29)
(915,)


In [23]:
X#[1][0]

array([[2.2493, 3.0027, 4.6685, ..., 0.    , 0.    , 1.    ],
       [5.126 , 9.9973, 7.0055, ..., 1.    , 1.    , 4.75  ],
       [1.0329, 1.0329, 1.4575, ..., 0.    , 0.    , 4.    ],
       ...,
       [8.4959, 8.4959, 9.0055, ..., 0.    , 1.    , 8.    ],
       [0.7589, 2.8329, 0.7589, ..., 0.    , 0.    , 1.    ],
       [3.1205, 3.1205, 4.0027, ..., 0.    , 0.    , 3.    ]])

In [17]:
y

array([1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
       1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1.,
       1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
       0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0.,
       1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
       0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1.,
       0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0.,
       1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
       0., 0., 0., 0., 1.

In [18]:
folds = StratifiedKFold(n_splits=10)

# Logistic Regression

In [24]:
lr = LogisticRegression()

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [40]:
fit = lr.fit(X_train, y_train)

In [41]:
fit.score(X_test, y_test)

0.6436363636363637

In [44]:
score_lr =[]

In [45]:
for train_index, test_index in folds.split(X,y):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    score_lr.append(get_score(LogisticRegression() ,X_train, X_test, y_train, y_test))  
    

In [46]:
score_lr

[0.6521739130434783,
 0.6413043478260869,
 0.6413043478260869,
 0.6413043478260869,
 0.6413043478260869,
 0.6483516483516484,
 0.6483516483516484,
 0.6483516483516484,
 0.6483516483516484,
 0.6483516483516484]

# Random Forest

In [47]:
rf=RandomForestClassifier(n_estimators=40)

In [23]:
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=40)

In [27]:
rf.score(X_test, y_test)

0.9127272727272727

In [49]:
cross_val_score(LogisticRegression(), X, y, cv=10)

array([0.65217391, 0.64130435, 0.64130435, 0.64130435, 0.64130435,
       0.64835165, 0.64835165, 0.64835165, 0.64835165, 0.64835165])